In [1]:
import re
import jieba as jb
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchnlp.encoders.text import StaticTokenizerEncoder

### 数据准备

In [2]:
# 设置最频繁使用的152000个词
MAX_NB_WORDS = 152000

# 每条cut_keyword最大的长度50
MAX_SEQUENCE_LENGTH = 50

# 设置Embedding层的维度
EMBEDDING_DIM = 128

# 模型参数的保存地址
model_path = "lstm.pth"

# 注词表的地址
vocab_path = "vocab.txt"

In [3]:
vocab = []
with open(vocab_path, "r") as file:
    for eachline in file:
        vocab.append(eachline[:-1])

divided_data = pd.read_csv("keyword_counts.csv", encoding="utf-8", index_col=0)

In [4]:
tok = StaticTokenizerEncoder(sample=vocab[1:], reserved_tokens=['<pad>'])  # 初始化标注器

cat_to_chinese = {
    'A': '马克思主义、列宁主义、毛泽东思想、邓小平理论',
    'B': '哲学、宗教',
    'C': '社会科学总论',
    'D': '政治、法律',
    'E': '军事',
    'F': '经济',
    'G': '文化、科学、教育、体育',
    'H': '语言、文字',
    'I': '文学',
    'J': '艺术',
    'K': '历史、地理',
    'N': '自然科学总论',
    'O': '数理科学和化学',
    'P': '天文学、地球科学',
    'Q': '生物科学',
    'R': '医药、卫生',
    'S': '农业科学',
    'T': '工业技术',
    'U': '交通运输',
    'V': '航空、航天',
    'X': '环境科学、安全科学',
    'Z': '综合性图书'
}

id_to_cat = {
    0: 'R', 1: 'S',
    2: 'F', 3: 'C',
    4: 'A', 5: 'D',
    6: 'H', 7: 'T',
    8: 'O', 9: 'K',
    10: 'I',11: 'B',
    12: 'E',13: 'G',
    14: 'J',15: 'N',
    16: 'P',17: 'Q',
    18: 'U',19: 'V',
    20: 'X'
}

### 定义网络

In [5]:
class LSTMnet(nn.Module):
    def __init__(self, output_size, hidden_dim, embedding_dim, bidirectional):
        super(LSTMnet, self).__init__()
        self.output_size = output_size
        self.liner1_input_size = hidden_dim * 2 if bidirectional else hidden_dim
        self.Embedding = nn.Embedding(MAX_NB_WORDS, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=0.2, bidirectional=bidirectional,
                            num_layers=2)
        self.dropout1 = nn.Dropout(0.25)
        self.linear1 = nn.Linear(self.liner1_input_size, 64)
        self.dropout2 = nn.Dropout(0.3)
        self.linear2 = nn.Linear(64, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = x.t()
        x = self.Embedding(x)
        lstm_out, hidden = self.lstm(x)
        lstm_out = lstm_out[-1]
        out = self.dropout1(lstm_out)
        out = self.linear1(out)
        out = self.dropout2(out)
        out = self.linear2(out)
        out = self.sig(out)
        return out

### 预测部分

In [6]:
# 定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip() == '':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('', line)
    return line


def stopwords_list(filepath):
    ret = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return ret


stopwords = stopwords_list("chineseStopWords.txt")

In [7]:
def lstm_pred(inputx,epochs=10):
    inputx = " ".join(inputx)
    inputx = tok.encode(inputx)
    inputx = np.array([np.array(inputx)])
    inputx = torch.from_numpy(inputx)
    
    model = LSTMnet(output_size=21,
                    hidden_dim=50,
                    embedding_dim=EMBEDDING_DIM,
                    bidirectional=True)
    
    model.load_state_dict(torch.load(model_path))
    
    pred = np.zeros(21)
    for _ in range(epochs):
        out = model(inputx)
        pred += out.detach().numpy()[0]

    pred /= epochs
    
    great_arg = [i for i in pred.argsort()[-3:][::-1]]
    
#     mid_out = ['\t{}.{} :\n\t\t {:.15f}\n'.format(ix+1, cat_to_chinese[id_to_cat[int(i)]], pred[int(i)]) for ix, i in enumerate(great_arg)]
#     print('输入文段的类别概率为\n' + " ".join(mid_out))
    
    flag = True if pred[int(great_arg[0])] -  pred[int(great_arg[1])] <= 0.21 else False
    
    return great_arg, flag

### 关键词提取

In [8]:
from TextRank import textRank


def text_reduce(text, n=12):
    text = remove_punctuation(text)
    text = [w for w in list(jb.cut(text)) if w not in stopwords]
    text = [w for w in text if w in divided_data.index]
    
    text = " ".join(text)
    
    text_reduce = textRank.TextRank(text, pr_config={'alpha':0.85, 'max_iter':100})
    reduced = text_reduce.get_n_keywords(n)
    
    return [i[0] for i in reduced]


def predict(text, n=12, epochs=10):
    rw = text_reduce(text, n)
    arg,flag = lstm_pred(rw,epochs)
    
    put = '\nLSTM Out:判断该文段的标签应为:【{}】{}'.format(id_to_cat[int(arg[0])], cat_to_chinese[id_to_cat[int(arg[0])]])
    
    if flag:
        put += '\nLSTM Out:判断该文段有第二标签:【{}】{}'.format(id_to_cat[int(arg[1])], cat_to_chinese[id_to_cat[int(arg[1])]])
    
    print('关键词为：\n\t', rw)
    
    print(put)

    return None

In [9]:
# 分类号: J205
# 中图分类: 绘画评论、欣赏

words = "将一百多幅名气大、影响广、代表性强的世界名画辑录成册，为了帮助读者提高阅读效果，\
书中设置了“必知理由”、“名画档案”、“画家简介”、“名画欣赏”、“绘画知识”等栏目，\
从艺术、历史、文化等多个角度解读世界名画的精髓之所在，让读者深刻体会名画背后鲜为人知的故事，\
领略艺术大师们创作的艰辛，全方位把握西方艺术所蕴涵的人文精神。"

predict(words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MSN\AppData\Local\Temp\jieba.cache
Loading model cost 0.316 seconds.
Prefix dict has been built successfully.


关键词为：
	 ['名画', '艺术', '读者', '世界', '名气', '影响', '蕴涵', '艺术大师', '书', '创作', '效果', '领略']

LSTM Out:判断该文段的标签应为:【J】艺术


In [10]:
# 分类号: D8
# 中图分类: 外交、国际关系

words = "本书是言简意赅、观点犀利的大众化读本，总结、评价以美国为主导的国际政治秩序。书中点评到的绝大多数议题包含着强烈的刺激性，\
并和当今的国际政治经验、秩序密切相关——美国政府海外政策的关注点是什么？其干涉世界事务的权力合法性如何？\
国际恐怖主义的主要源头真的是以美国为首的资本主义列强吗？反恐背景下，美国巨无霸式的工商企业是如何影响政府决策的？\
在全球战略的层面上，支持干涉计划、制造流血冲突且从中牟取暴利的幕后财团到底是怎么回事？这一切的背后，还有哪些力量掌控着今日世界的秩序？\
极权横行的现状里，现代传媒、知识分子和普通民众的相互关系如何，又该怎样行事？"

predict(words)

关键词为：
	 ['美国', '秩序', '国际', '干涉', '世界', '关系', '观点', '政治', '民众', '大众化', '普通', '知识分子']

LSTM Out:判断该文段的标签应为:【D】政治、法律
LSTM Out:判断该文段有第二标签:【B】哲学、宗教


In [11]:
# 分类号: E0
# 中图分类: 军事理论

words = "《牛津军事心理学（中文翻译版）》共分27章，涉及临床心理学、普通心理学在军队特殊领域的应用、人机工效与组织心理学、\
实验心理学和社会心理学五大类。《牛津军事心理学（中文翻译版）》内容十分丰富，从新兵的招募、训练、社会化和分配，\
到对军官的选拔、任用、激励、管理与社会支持，还有教育、矫正和治疗等。不仅对军事心理学在各领域的历史沿革情况做了系统的介绍，\
而且突出反映了当代军事心理学最关注的前沿问题和最新研究进展，是一本体现时代特点、内容翔实的军事心理学小百科全书。"

predict(words)

关键词为：
	 ['心理学', '军事', '内容', '领域', '中文翻译', '任用', '选拔', '激励', '军官', '管理', '分配', '社会']

LSTM Out:判断该文段的标签应为:【C】社会科学总论


In [12]:
# 分类号: E123
# 中图分类: 干部制度、人事
words = "《158个国家军队授衔内幕》主要内容：当今世界有哪些国家实行了军衔制？各国军衔是如何编制的？\
有什么不同？美国、俄罗斯、英国、法国、日本等大国，它们的军衔历史和现状是怎样的？如何识别和区分各国的军衔标志？\
军衔与职务是一种什么样的关系？军人退役后还能挂军衔吗？……这些鲜为人知的问题，《158个国家军队授衔内幕》将一一为您揭开。"

predict(words)

关键词为：
	 ['军衔', '国家', '内幕', '英国', '俄罗斯', '法国', '军人', '美国', '日本', '内容', '关系', '退役']

LSTM Out:判断该文段的标签应为:【E】军事


In [13]:
# 分类号: O151.2
# 中图分类: 线性代数
words = "《线性代数学习与考研指导》主要内容：线性代数是高等学校理工科专业普遍开设的一门重要的基础理论课程，\
是构建大学生数学知识结构中的重要一环。该课程的特点是理论性强，计算量大，解题技巧繁杂。为了帮助学生学好这门课程，\
我们组织具有多年教学经验的优秀教师编写了《线性代数学习与考研指导》，旨在帮助同学们澄清一些容易混淆的概念，掌握解题技巧，拓展视野，提高能力。"

predict(words)

关键词为：
	 ['课程', '线性代数', '解题技巧', '帮助', '指导', '提高', '视野', '经验', '教学', '混淆', '专业', '优秀教师']

LSTM Out:判断该文段的标签应为:【O】数理科学和化学


In [14]:
data = pd.read_csv("../dataset26p_cutkeyword.csv", encoding="utf8")
data

cat_id                                        cut_keyword
0            0  道家 文化 中医学 道家 文化 研究 中医学 医学史 研究 本书 充足 史料 严密 逻辑 证...
1            0  古典艺术 中医学 中国 医药学 研究 艺术史 研究 中国 古代 本书 中国 古典艺术 源流 ...
2            0  甲骨 文化 中医学 中国 医药学 本书 共章 甲骨 文化 概述 甲骨文 中 医学 基础知识 ...
3            0  中国 中医 文化 传播 史 中国 医药学 文化 传播 医学史 中国 古代 本书 共 分为 十...
4            0  第三次 医学 革命 理论 思维 医学 伦理学 研究 本书 共 分为 医学 历史学 医学 哲学...
...        ...                                                ...
133420      20  全国 注册 安全 工程师 执业资格 考试 红宝书 安全 生产 管理 知识 安全 生产 生产 ...
133421      20  安全 知识 知道 户外 安全 安全 教育 儿童读物 本书 主要 内容 包括 不要 铁轨 玩 ...
133422      20  二十一 世纪 安全 科学 技术 发展趋势 安全 科学 学科 文集 本书 中国科协 次 青年 ...
133423      20  事故 调查 分析 技术 事故 学科 调查 学科 高等学校 事故 分析 学科 高等学校 教材 ...
133424      20  安全 系统工程 第版 安全 系统工程 高等学校 教材 本书 煤炭工业 出版社 十二五 规划 ...

[133425 rows x 2 columns]

In [15]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

def reduced(line):
    text = remove_punctuation(line)
    text = [w for w in list(jb.cut(text)) if w not in stopwords]
    text = [w for w in text if w in divided_data.index]
    
    if len(text) <= 1:
        print(line, text)
        return line
        
    text = " ".join(text)
    
    ret = None
    try:
        text_reduce = textRank.TextRank(text, pr_config={'alpha':0.85, 'max_iter':10})
        reduced = text_reduce.get_n_keywords(12)
        ret = " ".join([i[0] for i in reduced])
    except:
        ret = line
        print("E", line, text)

    return ret
data['cut_keyword'].apply(reduced)

医学史 ['医学史']
社会医学 ['社会医学']
医学史 ['医学史']
仲景 学术论坛 ['学术论坛']
如东县 卫生 志下 ['卫生']
乳癌 乳腺癌 ['乳腺癌']
齐民要术 ['齐民要术']
中国 农书下 ['中国']
夏小正 疏义 ['夏小正']
泛腾 之书辑释 ['之书辑释']
农话 []
农桑辑 ['农桑辑']
齐民要术 ['齐民要术']
斋民 要术 ['要术']
农桑辑 ['农桑辑']
齐民要术 ['齐民要术']
农桑辑 要卷 ['农桑辑']
古农书 ['古农书']
齐民要术 ['齐民要术']
泛胜 之书辑释 ['之书辑释']
农桑辑 要卷 ['农桑辑']
齐民要术 ['齐民要术']
农杂 辑要 ['辑要']
金芳备 祖 ['祖']
绿色食品 ['绿色食品']
泛胜 之书辑释 ['之书辑释']
齐民要术 ['齐民要术']
金薯 传习录 ['传习录']
农桑辑 要卷 ['农桑辑']
油料作物 ['油料作物']
夏小正 疏义 ['夏小正']
土壤胶体 第册 ['第册']
指示植物 []
打坝 淤地坝 ['淤地坝']
农家肥料 有机肥料 ['有机肥料']
洪雅 土壤 ['土壤']
真空 井 喷灌 喷溉 ['真空']
污水 灌田 ['污水']
农业机械 ['农业机械']
土地平整 整地 ['整地']
作物栽培 ['作物栽培']
农学 ['农学']
除虫菊 第版 ['第版']
水耕 栽培 ['栽培']
稻热 瘟病 ['瘟病']
长绒棉 棉花 长绒棉 ['棉花']
紫胶虫 寄主植物 紫胶虫 ['寄主植物']
养鱼 ['养鱼']
幽燕 义侠 ['义侠']
海灯 法师 ['法师']
海灯 法师 ['法师']
天盘 梁歼 匪记 ['梁歼']
杨靖宇 将军 ['将军']
桥隆 飙下 ['飙下']
捕虎记 []
赵一曼 []
鱼鹰 姑娘 ['姑娘']
夜渡 小清河 ['小清河']
红红的 雨花石 ['雨花石']
桥隆 飙上 ['飙上']
智炸 军火库 ['军火库']
日莲 []
大钉靴 奇闻 ['奇闻']
盐民 游击队 ['游击队']
探索 ['探索']
茉莉花 传奇 ['传奇']
茉莉花 传奇 ['传奇']
雷声 阵阵 ['阵阵']
变天 记之四破 敌堡 ['记之四破']
活捉 胡司令 ['胡司令']
智歼 眼镜蛇 ['眼镜蛇']
智截军 火车 ['火车']
港九 枪

细胞学 ['细胞学']
遗传学 ['遗传学']
遗传学 ['遗传学']
遗传学 ['遗传学']
遗传 变异 ['遗传变异']
通径 分析 ['分析']
生物化学 ['生物化学']
教海 拾贝 ['拾贝']
生理学 ['生理学']
红雨 青山 ['青山']
生理学 试题 答案 ['生理学']
性性 []
成长 ['成长']
化学 ['化学']
氨基酸 ['氨基酸']
生物化学 实难 ['生物化学']
组知 化学 ['化学']
酵素 ['酵素']
蛇岛 []
微生物学 ['微生物学']
微生物学 ['微生物学']
益菌 害菌 ['害菌']
微生物 ['微生物']
蟋蟀 赏玩 ['蟋蟀']
蠓科 概论 ['概论']
紫胶 []
业务学习 资料 ['资料']
铁路运输 ['铁路运输']
接触网 ['接触网']
接触网 ['接触网']
城市道路 ['城市道路']
全是 特种车 ['全是']
全是 大卡车 ['全是']
全是 小汽车 ['全是']
汽车 ['汽车']
水翼船 []
航海 ['航海']
天外来客 ['天外来客']
寻找 ['寻找']
神秘 ['神秘']
耐腐蚀 用钢 ['用钢']
玻璃钢 ['玻璃钢']
大全 ['大全']
塔器 译文集 ['译文集']
外星人 ['外星人']
胶带 输送机 ['输送机']
工业化学 []
相律 反应 ['反应']
球型 贮罐 ['贮罐']
喷射飞机 动力系统 ['动力系统']
直升机 ['直升机']
飞机 ['飞机']
飞机 ['飞机']
第版 ['第版']
战斗机 ['战斗机']
大疆 无人机 ['无人机']
喷射客机 []
飞将军 []
航天飞机 ['航天飞机']
公路运输 ['公路运输']
太空梭 科学 ['科学']
义胆照 青史 ['青史']
太空 揽胜 ['太空']
航天技术 ['航天技术']
年报 ['年报']
废热 回收 ['回收']
废液 不废 ['废液']


0                 道家 研究 中医学 技术 保健 治疗 共用 医 介绍 出发 文化 原则
1             文化 中医 中国 古典艺术 研究 起到 国际交流 增进 价值 艺术 开拓 尝试
2               甲骨 甲骨文 文化 进行 医学 临床 中医学 综合 考证 整理 研究 全面
3               中国 传播 时期 中医 文明 文化 分为 内容 古代 包括 卫生 原始社会
4                 医学 伦理学 动机 牟利 杜 名医 台湾 包括 内容 理论 革命 思维
                             ...                     
133420          安全 考点 考试 诠释 工程师 知识 化解 管理 生产 剖析 精选 知识点
133421        安全 车窗 打冲锋 十字路口 踢球 玩 包括 内容 儿童读物 教育 知识 知道
133422       安全 技术 科研成果 青年 科学家 科学技术 取得 领域 收 论文集 论文 报告
133423    事故 分析 技术 学科 高等学校 调查 后果 火灾 故障诊断 重大事故 模拟 分析方法
133424      安全 教材 章节 系统工程 分析方法 系统 修改 上作 增加 案例 全面 具体内容
Name: cut_keyword, Length: 133425, dtype: object

In [21]:
data.loc[133125]

cat_id                                                        20
cut_keyword    防火 防爆 知识 防火 学科 基本知识 防爆 学科 基本知识 本书 主要 阐述 工业 企业 ...
Name: 133125, dtype: object